<a href="https://colab.research.google.com/github/lukilearn/CAPSTONE_2_Bike_Sharing_Demand_Prediction_ML_Regression/blob/main/Bike_Sharing_Demand_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    -



##### **Project Type**    - REGRESSION
##### **Contribution**    - Individual
##### **Name -** G MOHAMED LUQMAN


# **Project Summary -**

Write the summary here within 500-600 words.

# **GitHub Link -**

Provide your GitHub Link here.

# **Problem Statement**


**Write Problem Statement Here.**

# **General Guidelines** : -  

1.   Well-structured, formatted, and commented code is required.
2.   Exception Handling, Production Grade Code & Deployment Ready Code will be a plus. Those students will be awarded some additional credits.
     
     The additional credits will have advantages over other students during Star Student selection.
       
             [ Note: - Deployment Ready Code is defined as, the whole .ipynb notebook should be executable in one go
                       without a single error logged. ]

3.   Each and every logic should have proper comments.
4. You may add as many number of charts you want. Make Sure for each and every chart the following format should be answered.
        

```
# Chart visualization code
```
            

*   Why did you pick the specific chart?
*   What is/are the insight(s) found from the chart?
* Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

5. You have to create at least 15 logical & meaningful charts having important insights.


[ Hints : - Do the Vizualization in  a structured way while following "UBM" Rule.

U - Univariate Analysis,

B - Bivariate Analysis (Numerical - Categorical, Numerical - Numerical, Categorical - Categorical)

M - Multivariate Analysis
 ]





6. You may add more ml algorithms for model creation. Make sure for each and every algorithm, the following format should be answered.


*   Explain the ML Model used and it's performance using Evaluation metric Score Chart.


*   Cross- Validation & Hyperparameter Tuning

*   Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

*   Explain each evaluation metric's indication towards business and the business impact pf the ML model used.




















# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:

# Import Libraries
import pandas as pd
import numpy as np
from datetime import datetime as dt


# Import Visualization Libraries
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Import warnings
import warnings
warnings.filterwarnings('ignore')

# Import preporcessing libraries
from sklearn.preprocessing import MinMaxScaler,StandardScaler

# Import model selection libraries
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV

# Import Outlier influence library
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Import Model
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from xgboost import XGBRegressor
import xgboost as xgb

# Import evaluation metric libraries
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

# Import tree for visualization
from sklearn.tree import export_graphviz
from sklearn import tree
from IPython.display import SVG,display
from graphviz import Source

### Dataset Loading

In [ ]:

# Load Dataset from github repository
url = 'https://raw.githubusercontent.com/lukilearn/CAPSTONE_2_Bike_Sharing_Demand_Prediction_ML_Regression/main/SeoulBikeData.csv'
data = pd.read_csv(url, encoding='unicode_escape')

### Dataset First View

In [ ]:
# Dataset First Look

# View top 5 rows of the dataset
data.head()

In [ ]:
# View last 5 rows of the dataset

data.tail()

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count

# Checking number of rows and columns of the dataset using shape
print("Number of rows are: ",data.shape[0])
print("Number of columns are: ",data.shape[1])

### Dataset Information

In [ ]:
# Dataset Info

data.info()



#### Duplicate Values, Missing Values/Null Values, Unique value

In [ ]:
# Dataset Duplicate Value Count
# This user define function will give the type,count of null and non null values as well as null ratio

def bike_info():
  temp=pd.DataFrame(index=data.columns)
  temp["datatype"]=data.dtypes
  temp["not null values"]=data.count()
  temp["null value"]=data.isnull().sum()
  temp["% of null value"]=data.isnull().mean()*100
  temp["unique count"]=data.nunique() # Return Series with number of distinct elements. Can ignore NaN values.
  return temp
bike_info()

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count

import missingno as msno
msno.bar(data, color='green',sort='ascending', figsize=(10,2), fontsize=15)

In [ ]:

# Visualizing the missing values using Heatmap
plt.figure(figsize=(12,2))
sns.heatmap(data.isna(), cmap = 'coolwarm')

### What did you know about your dataset?

There are a total of 14 feature columns where Rented Bike Count is the dependent variable column. The total number of observations(rows) are 8760.

There are no duplicate rows in the dataset.

Also there are no missing values or Null values in the dataset.

Dataset has all unique values i.e., there is no duplicate, which means data is free from bias as duplicates which can cause problems in downstream analysis, such as biasing results or making it difficult to accurately summarize the data.
Date has some object data types, it should be datetime data type.

## ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns

print(f'Features: {data.columns.to_list()}')

In [ ]:

#Looking for the description of the dataset to get insights of the data
data.describe()

### Variables Description

 The dataset contains weather information (Temperature, Humidity, Windspeed, Visibility, Dewpoint, Solar radiation, Snowfall, Rainfall), the number of bikes rented per hour and date information.

Attribute Information:

    Date : year-month-day
    Rented Bike count - Count of bikes rented at each hour
    Hour - Hour of the day
    Temperature-Temperature in Celsius
    Humidity - %
    Windspeed - m/s
    Visibility - 10m
    Dew point temperature - Celsius
    Solar radiation - MJ/m2
    Rainfall - mm
    Snowfall - cm
    Seasons - Winter, Spring, Summer, Autumn
    Holiday - Holiday/No holiday
    Functional Day - NoFunc(Non Functional Hours), Fun(Functional hours)

### Observations:

    We are focusing on several key columns of our dataset, including 'Hour', 'Holiday', 'Functioning Day', 'Rented Bike Count', 'Temperature(°C)', and 'Seasons', as they contain a wealth of information.
    By utilizing these features, we plan to create a regression model and implement various regression algorithms.
    There is a column 'Hour' which might be considered a categorical feature or maybe a numerical feature based on the data we will try both and see the result difference.


### Check Unique Values for each variable.

In [ ]:

# Check Unique Values for each variable using a for loop
for i in data.columns.tolist():
  print("No. of unique values in",i,"is",data[i].nunique())

## 3. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
# creating a some new features:

# Renaming complex columns name for the sake of simplicity
data=data.rename(columns={'Rented Bike Count':'rented_bike_count',
                                'Date':'date',
                                'Hour':'hour',
                                'Seasons':'seasons',
                                'Holiday':'holiday',
                                'Temperature(°C)':'temperature',
                                'Humidity(%)':'humidity',
                                'Wind speed (m/s)':'wind_speed',
                                'Visibility (10m)':'visibility',
                                'Dew point temperature(°C)':'dew_point_temperature',
                                'Solar Radiation (MJ/m2)':'solar_radiation',
                                'Rainfall(mm)':'rainfall',
                                'Snowfall (cm)':'snowfall',
                                'Functioning Day':'func_day'})

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
# Splitting Date into year, month, day & day_name
data.date = pd.to_datetime(data.date)

data['day'] = data['date'].dt.day
data['month'] =data['date'].dt.month
data['year'] = data['date'].dt.year
data['weekday'] = data['date'].dt.day_name()

# droping Date column
data.drop('date', axis=1, inplace=True)

In [ ]:
data.hour.unique()

The hours of the day follow a clear sequence, with 9 am being closer to 10 am than it is to 8 am, and farther from 6 pm. This feature can be classified as a discrete ordinal variable. We will consider the hour as a categorical value and transform it into a numerical value to see if there is any difference in the results.


In [ ]:
def session(x):

    '''
    For exploratory data analysis (EDA) purposes, the "Hour" column can be converted into categorical variables
    such as "Morning", "Noon", and "Night", without altering the existing label encoding format of the "Hour" column.
    This conversion is not necessary for model training.
    '''

    if x>4 and x<=8:
        return 'Early Morning'
    elif x>8 and x<=12:
        return 'Morning'
    elif x>12 and x<=16:
        return 'Afternoon'
    elif x>16 and x<=20:
        return 'Evening'
    elif x>20 and x<=24:
        return 'Night'
    elif x<=4:
        return 'Late Night'

#apply funtion to make new category column
data['session'] = data['hour'].apply(session)

In [ ]:
# defining continuous independent variables separately
data_var = ['temperature', 'humidity', 'wind_speed', 'visibility','dew_point_temperature','solar_radiation', 'rainfall', 'snowfall']

In [ ]:
#defining dependent variable
dependent_variable = ['rented_bike_count']

# defining categorical independent variables separately
cat_var = ['hour','seasons', 'holiday', 'func_day', 'month', 'weekday']

In [ ]:
cat_var


In [ ]:
data.head()

### What all manipulations have you done and insights you found?

From the Date column, 'month' and 'day of the week' columns are created.

To capture this trend, we can define a new feature 'weekend' which indicates whether a said day is a weekend (1) or not (0).

From the day of the week column, weekend column is created where 6 and 7 are the weekends (Saturday and Sunday).

We have also defined the continuous variables, dependent variable and categorical variables for ease of plotting graphs.

## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

# **Exploratory Data Analysis Of The Data Set**

**Why do we perform EDA?**
* ***An EDA is a thorough examination meant to uncover the underlying structure of a data set and is important for a company because it exposes trends, patterns, and relationships that are not readily apparent.***

**Why do you do univariate analysis?**

* ***The key objective of Univariate analysis is to simply describe the data to find patterns within the data.***

**Why do you do Bivariate analysis?**
* ***The key objective of bivariate analysis is to explore and understand the relationship or association between two variables***

**Why do you do multivariate analysis?**
* ***The key objective of multivariate analysis is to study and understand the relationships, patterns, and interactions among multiple variables simultaneously.***

#### Chart - 1 : Dependent variable Distribution

In [ ]:
# Chart-1 Visualization code for distribution of target variable
plt.figure(figsize=(10,8))
sns.distplot(data['rented_bike_count'])
plt.show()

##### **1.Why did you pick the specific chart?**

A distplot, also known as a histogram-kernel density estimate (KDE) plot. It is useful because it provides a quick and easy way to check the distribution of the data, identify patterns or outliers, and compare the distribution of multiple variables. It also allows to check if the data is following normal distribution or not.

Thus, I used the histogram plot to analyse the variable distributions over the whole dataset whether it's symmetric or not.

##### 2. **What is/are the insight(s) found from the chart?**

From above distribution plot of dependent variable rented bike, we can clearly see that the distribution is positively skewed (Right skewed).

It means that distribution is not symmetric around the the mean.


##### 3. **Will the gained insights help creating a positive business impact?**
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, definately from this insight we got to know that we our data is not normally distributed so, before doing or implementing any model on this data we need to normalise this data.

#### Chart - 2  Distribution/ Box plot

In [ ]:
# Chart - 2 : Visualizing code of histogram plot & boxplot for each columns to know the data distribution
for col in data.describe().columns:
    fig,axes = plt.subplots(nrows=1,ncols=2,figsize=(18,6))
    sns.histplot(data[col], ax = axes[0],kde = True)
    sns.boxplot(data[col], ax = axes[1],orient='h',showmeans=True,color='pink')
    fig.suptitle("Distribution plot of "+ col, fontsize = 15)
    plt.show()

##### **1. Why did you pick the specific chart?**

A histplot is a type of chart that displays the distribution of a dataset. It is a graphical representation of the data that shows how often each value or group of values occurs. Histplots are useful for understanding the distribution of a dataset and identifying patterns or trends in the data. It is also useful when dealing with large data sets (greater than 100 observations). It can help detect any unusual observations (outliers) or any gaps in the data.

Thus, we used the histogram plot to analysis the variable distributions over the whole dataset whether it's symmetric or not.

A boxplot is used to summarize the key statistical characteristics of a dataset, including the median, quartiles, and range, in a single plot. Boxplots are useful for identifying the presence of outliers in a dataset, comparing the distribution of multiple datasets, and understanding the dispersion of the data. They are often used in statistical analysis and data visualization.

Thus, for each numerical varibale in the given dataset, we used box plot to analyse the outliers and interquartile range including mean, median, maximum and minimum value.

##### 2. **What is/are the insight(s) found from the chart?**

From above univariate analysis of all continuous feature variables. We got to know that only tempture and humidity columns are looks normally distributed others shows the different distributions.

Also we can see that there are outlier values in snowfall, rainfall, wind speed & solar radiation columns


##### 3. **Will the gained insights help creating a positive business impact?**

Are there any insights that lead to negative growth? Justify with specific reason.

Histogram and Box plot cannot give us whole information regarding data. It's done just to see the distribution of the column data over the dataset.

#### Chart - 3 : Dependent variable with continuous variables (Bivariate)

In [ ]:
# Chart - 3

# Analyzing the relationship between the dependent variable and the continuous variables
for i in data_var:
  plt.figure(figsize=(11,8))
  sns.regplot(x=i,y=dependent_variable[0],data=data)
  plt.xlabel(i)
  plt.ylabel(dependent_variable[0])
  plt.title(i+' vs '+ dependent_variable[0])
  plt.show()

##### **1. Why did you pick the specific chart?**

Regplot is used to create a scatter plot with linear regression line. The purpose of this function is to visualize the relationship between two continuous variables. It can help to identify patterns and trends in the data, and can also be used to test for linearity and independence of the variables.

To check the patterns between independent variable with our rented bike dependent variable we used this regplot.


##### **2. What is/are the insight(s) found from the chart?**

From above regression plot we can see that there is some linearity between temperature, solar radiation & dew point temperature with dependent variable rented bike

Other variables are not showing any patterns.


##### 3. **Will the gained insights help creating a positive business impact?**
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, it helped a little bit from this we got to know that there are few variables which are showing some patterns with dependent variable this variable are maybe important feature while predicting for rented bike count so business needs focus on these variables.

#### **Chart - 4 : Categorical variables with dependent variable (bivariate)**

In [ ]:
# Chart - 4 Analyzing the relationship between the dependent variable and the categorical variables
for i in cat_var:
  plt.figure(figsize=(11,8))
  sns.barplot(x=i,y=dependent_variable[0],data=data)
  plt.xlabel(i)
  plt.ylabel(dependent_variable[0])
  plt.title(i+' vs '+ dependent_variable[0])
  plt.show()

##### **1. Why did you pick the specific chart?**

Bar charts are used to compare the size or frequency of different categories or groups of data. Bar charts are useful for comparing data across different categories, and they can be used to display a large amount of data in a small space.

To show the distribution of the rented bike count with other categorical variables we used bar charts.


##### **2. What is/are the insight(s) found from the chart?**

From above bar charts we got insights:

1. In hour vs rented bike chart there is high demand in the morning 8'o clock and evening 18'o clock
2. From season vs rented bike chart there is more demand in summer and less demand in winter.
3. There is high demand on working days.
4. From month chart we know that there is high demand in month of june.


##### **3. Will the gained insights help creating a positive business impact?**
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, this insights are going to provide some positive business impact, beacause analysing the demand on the basis of categorical varible we got know that when demand for bike is more so we can focus more on that portion.



#### Chart - 5 : Rented Bike vs Hour

In [ ]:
# Chart - 5 visualization code

#ploting line graph
# group by Hrs and get average Bikes rented, and precent change
avg_rent_hrs = data.groupby('hour')['rented_bike_count'].mean()

# plot average rent over time(hrs)
plt.figure(figsize=(12,6))
sns.lineplot(data=avg_rent_hrs, marker='o')
plt.title('Average bike rented per hour')

##### **1. Why did you pick the specific chart?**

A line plot, also known as a line chart or line graph, is a way to visualize the trend of a single variable over time. It uses a series of data points connected by a line to show how the value of the variable changes over time.

Line plots are useful because they can quickly and easily show trends and patterns in the data. They are particularly useful for showing how a variable changes over a period of time. They are also useful for comparing the trends of multiple variables.

To see how rented bike demand is distributed over 24 hours time we used line plot.


##### **2. What is/are the insight(s) found from the chart?**

From above line plot we can clearly see that there is high demand in the morning and in the evening.

##### **3. Will the gained insights help creating a positive business impact?**
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, from above insight we know that there is high demand in morning and evening so business needs to focus more on that time slot. as well as try to meet the demand on that time slot.


#### Chart - 6 : Bike demand throughout the day (Multivariate)

In [ ]:
# Chart - 6 visualization code

for i in cat_var:
  if i == 'hour':
    continue
  else:
    fig, ax = plt.subplots(figsize=(12,8))
    sns.pointplot(data=data, x='hour', y='rented_bike_count', hue=i, ax=ax)
    plt.title('Hourly bike demand broken down based on the attribute: '+i)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left',title=i)
    plt.show()

##### **1. Why did you pick the specific chart?**

A line plot, also known as a line chart or line graph, is a way to visualize the trend of a single variable over time. It uses a series of data points connected by a line to show how the value of the variable changes over time.

Line plots are useful because they can quickly and easily show trends and patterns in the data. They are particularly useful for showing how a variable changes over a period of time. They are also useful for comparing the trends of multiple variables.

To show the demand of rented bike throughout the day on the basis of other categorical variable we used line plot drawing multiple lines on charts.

##### **2. What is/are the insight(s) found from the chart?**

From above line plots we see that :

1. In winter season there is no significant demand even in the morning or in the evening.
2. On the functional day (i.e No Holiday) there is spike in morning and in evening, but that is not there on Holidays.
3. Around 3 months in winter season (i.e December, January & February) there is low demand.
4. On weekend almost throught the day there is demand.


##### **3. Will the gained insights help creating a positive business impact?**
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, from this analysis we figure out some key factors such as high demand in morning and evening slot in all the seasons.

#### Chart - 7 : Categorical plot for seasons

In [ ]:

#plot for rented bike count seasonly
sns.catplot(x='seasons',y='rented_bike_count',data=data)

##### 1. Why did you pick the specific chart?

Catplot is used to create a categorical plot. Categorical plots are plots that are used to visualize the distribution of a categorical variable. They can be used to show how a variable is related to a categorical variable and can also be used to compare the distribution of multiple categorical variables.

To see the distribution of the rented bike on basis of season column we used catplot.

##### 2. What is/are the insight(s) found from the chart?

From above catplot we got know that:

1. There is low demand in winter
2. Also in all seasons upto the 2500 bike counts distribution is seen dense.


##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, from this catplot we know that there is high bike count upto the 2500 so, above that there maybe outliers present. business needs to evaluate that.

#### Chart - 8

In [ ]:

# Chart - 8 visualization code
Winter=data[data["seasons"]=='Winter'].sum()
Spring=data[data["seasons"]=='Spring'].sum()
Summer=data[data["seasons"]=='Summer'].sum()
Autumn=data[data["seasons"]=='Autumn'].sum()

BikeSeasons={"Winter":Winter["rented_bike_count"],"Spring":Spring["rented_bike_count"],"Summer":Summer["rented_bike_count"],"Autumn":Autumn["rented_bike_count"]}
plt.gcf().set_size_inches(10,10)
plt.pie(BikeSeasons.values(),labels=BikeSeasons.keys(), autopct='%1d%%');
plt.title("Repartition of bikes rental by season", fontsize=20)
plt.show()

##### 1. Why did you pick the specific chart?

Pie charts are generally used to show the proportions of a whole, and are especially useful for displaying data that has already been calculated as a percentage of the whole.

So, we used pie chart to see percentage distribution of rented bike on the basis of sseasons

##### 2. What is/are the insight(s) found from the chart?

From above pie chart:

1. In year data season summer contributes around 36% then autumn around 29%
2. Lowest demand in winter, it contributes around only 7%

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

This insights only tell about percentage contribution of year data of season varible, which clearly gave indication about demand.

#### Chart - 9 - Correlation Heatmap

In [ ]:
# Chart - 9 visualization code

corr = data.corr()
mask = np.zeros_like(corr)

mask[np.triu_indices_from(mask)] = True

with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(18, 9))
    ax = sns.heatmap(corr , mask=mask, vmin = -1,vmax=1, annot = True, cmap="YlGnBu")

##### 1. Why did you pick the specific chart?

The correlation coefficient is a measure of the strength and direction of a linear relationship between two variables. A correlation matrix is used to summarize the relationships among a set of variables and is an important tool for data exploration and for selecting which variables to include in a model. The range of correlation is [-1,1].

Thus to know the correlation between all the variables along with the correlation coeficients, we have used correlation heatmap.


##### 2. What is/are the insight(s) found from the chart?

From above correlation map we can clearly see that:

1. There is high multicolinearity between independent variable (i.e temperature & dew point temp, humidity & dew point temp, weekend & day of week).
2. There is correlation of temperature, hour, dew point temp & solar radiation with dependent variable rented bike.
3. Other than that we didnt see any correlation.

#### Chart - 10 - Pair Plot

In [ ]:
# Chart - 10 visualization code

# Pair Plot
sns.pairplot(data)
plt.show()

##### 1. Why did you pick the specific chart?

A pairplot, also known as a scatterplot matrix, is a visualization that allows you to visualize the relationships between all pairs of variables in a dataset. It is a useful tool for data exploration because it allows you to quickly see how all of the variables in a dataset are related to one another.

Thus, we used pair plot to analyse the patterns of data and realationship between the features. It's exactly same as the correlation map but here you will get the graphical representation.

##### 2. What is/are the insight(s) found from the chart?

From above pair plot we got to know that, there is not clear linear relationship between variables. other than dew point temp, temperature & solar radiation there is not any reationship.

## ***5. Hypothesis Testing***

### Based on your chart experiments, define three hypothetical statements from the dataset. In the next three questions, perform hypothesis testing to obtain final conclusion about the statements through your code and statistical testing.

Based on above chart experiments we have noticed that our dependent variable does not seems to normally distributed so we have made hypothetical assumption that our data is normally distributed and for that we have decided to do statistical analysis.

### Hypothetical Statement - 1

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

## Normality test

for normality test we decided
1. Null hypothesis : Data is normally distributed
2. Alternate hypothesis : Data is not normally distributed

#### 2. Perform an appropriate statistical test.

In [ ]:

# Perform Statistical Test to obtain P-Value
from scipy.stats import shapiro
test_data = data['rented_bike_count']

stats,p  = shapiro(test_data)
print('stats = %.2f, p=%.3f' % (stats,p))

if p <= 0.05:
  print("Reject null hypothesis i.e  Data is not normally distributed")
else:
  print('Accept null hypothesis i.e  Data is normally distributed')

##### Which statistical test have you done to obtain P-Value?

We use Shapiro-wilk statistical test to obtain the p-value and we got very less p-value which is less than 0.05.

##### Why did you choose the specific statistical test?

The Shapiro-Wilk test is used to test the normality of a sample. The test checks whether the sample data fits a normal distribution, which is often assumed for statistical analysis. The test results can help determine if the data should be transformed or if non-parametric statistical methods should be used instead of traditional parametric methods.

## ***6. Feature Engineering & Data Pre-processing***



What is data cleaning?

Data cleaning is the process of identifying and correcting or removing inaccuracies, inconsistencies, and missing values in a dataset. It is an important step in the data preparation process that ensures that the data is accurate, complete, and in a format that can be easily analyzed. Data cleaning may include tasks such as removing duplicate records, filling in missing values, correcting errors, and standardizing data formats. The goal of data cleaning is to improve the quality of the data and make it suitable for further analysis and modeling.





#### Duplicate Values


In [ ]:


# counting duplicate values
data.duplicated().sum()



### 1. Handling Missing Values

In [ ]:

# Missing Values/Null Values Count
print(data.isnull().sum())


#### Skewness

In [ ]:


# statistical summary
data.describe().T





As can be seen in the statistical summary for numerical features, there is a significant difference between the 75% percentile and maximum value, indicating that the dataset contains skewness and outliers.


In [ ]:
# removing rainfall and snowfall as it may remove important information as these 2 columns are highly skewed.
data_var.remove('rainfall')
data_var.remove('snowfall')

In [ ]:


# figsize
plt.figure(figsize=(15,6))
# title
plt.suptitle('Data Distibution of Numerical Features', fontsize=20, fontweight='bold', y=1.02)

for i,col in enumerate(data_var):
  plt.subplot(3,3,i+1)

  # dist plots
  sns.distplot(data[col])
  # x-axis label
  plt.xlabel(col)
  plt.tight_layout()






Observation:

1.  For numerical features, we can see that the majority of distributions are right-skewed. The distribution of rainfall, snowfall, and solar radiation is highly skewed to the right. It demonstrates that these columns have many outliers. Some columns are negatively skewed.
2.   Some of the variables can get a normal distribution when outliers are removed. As a result, it appears that outliers should be removed before the transformation. First, we will get rid of outliers, and then we check to see if we need to use the transformation technique again.





### 2. Handling Outliers

In [ ]:
# figsize
plt.figure(figsize=(15,5))
# title
plt.suptitle('Outlier Analysis of Numerical Features', fontsize=20, fontweight='bold', y=1)

for i,col in enumerate(data_var):
  plt.subplot(3,3,i+1)

  # countplot
  sns.boxplot(data[col],orient="h")
  # x-axis label
  plt.xlabel(col)
  plt.tight_layout()




Observation:


1.   Outliers are visible in the 'wind_speed','solar_radiation','rainfall',and 'snowfall' columns.
    Since we have limited datapoint hence we are not simply removing the outlier instead of that we are using the clipping method
2.   Clipping Method: In this method, we set a cap on our outliers data, which means that if a value is higher than or lower than a certain threshold, all values will be considered outliers. This method replaces values that fall outside of a specified range with either the minimum or maximum value within that range.




In [ ]:


# we are going to replace the datapoints with upper and lower bound of all the outliers

def clip_outliers(data):
    for col in data[data_var]:
        # using IQR method to define range of upper and lower limit.
        q1 = data[col].quantile(0.25)
        q3 = data[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr

        # replacing the outliers with upper and lower bound
        data[col] = data[col].clip(lower_bound, upper_bound)
    return data



In [ ]:


# using the function to treat outliers
data = clip_outliers(data)



In [ ]:
# checking the boxplot after outlier treatment

# figsize
plt.figure(figsize=(15,5))
# title
plt.suptitle('Outlier Analysis of Numerical Features', fontsize=20, fontweight='bold', y=1.02)

for i,col in enumerate(data_var):
  plt.subplot(3,3, i+1)

  # countplot
  sns.boxplot(data[col], orient='h')
  # x-axis label
  plt.xlabel(col)
  plt.tight_layout()


##### What all outlier treatment techniques have you used and why did you use those techniques?

Here we use IQR method and Capping method, Based on IQR method we set Upper limit and Lower limit of rented bike count and convert those outliers into median values.

Also we capp outliers upto 99th percentile and above that we convert those outliers into upper limit value

## Note :-

1. We have tried to remove the outliers but it is seen that there is drop in performance after removing the outliers around 10% drop in model performance

So, we have decided that we will perform the model without removing the outliers.

#### 3. Feature Manipulation & Selection

#### 1.Feature *Manipulation*

# Dew point temperature


We know that the dew point temperature is related to temperature and can be approximated as follows:

Td = T - ((100 - RH)/5.)

Where

    Td - Dew point temperature (in degrees Celsius)
    T - Observed temperature (in degrees Celsius)
    RH - Relative humidity (in percent)


In [ ]:


# Scatter plot to visualize the relationship between

plt.figure(figsize=(10,5))
plt.scatter(x='temperature',y='dew_point_temperature',data=data )
plt.xlabel('temperature')
plt.ylabel('dew_point_temp')
plt.title('Temperature VS Dew point Temperature')



In [ ]:

# correlation
data[['temperature','dew_point_temperature']].corr()



#### Observation:


    The temperature and dew point temperature are highly correlated (0.912).
    Also, from the above scatter plot, it is clear that as the temperature increases, the dew point temperature also increases.
    Hence we can drop the column from the dataset since it will not increase the accuracy of predictions, and will only increase the model complexity.



In [ ]:


# dropping dew point temperature
data.drop('dew_point_temperature', axis=1,inplace=True)



#### 2.Feature Selection

### **VIF**

A variance inflation factor(VIF) detects multicollinearity in regression analysis. Multicollinearity is when there’s correlation between predictors (i.e. independent variables) in a model; it’s presence can adversely affect your regression results. The VIF estimates how much the variance of a regression coefficient is inflated due to multicollinearity in the model.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):

  # For each X, calculate VIF and save in dataframe
  vif = pd.DataFrame()
  vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
  vif["features"] = X.columns

  return vif

In [ ]:
calc_vif(data[[i for i in data.describe().columns]])

Since the VIF factor of 'year' is too large hence we will remove the year from our data to build our model.

In [ ]:
# droping year column due to multi-collinearity

data.drop('year', axis=1, inplace=True)

# dropping columns those were created for EDA purpose only and do not account for any information addition
data.drop('weekday', axis=1, inplace=True)         # day column is already present with ordinal numbering
data.drop('session', axis=1, inplace=True)         # hour column is already present with ordinal numbering

In [ ]:
# Calculating the VIF again
calc_vif(data[[i for i in data.describe().columns]])

In [ ]:
# droping func_day column due to multi-collinearity

data.drop('func_day', axis=1, inplace=True)

In [ ]:
# Calculating the VIF again
calc_vif(data[[i for i in data.describe().columns]])

##### What all feature selection methods have you used  and why?


We have used pearson correlation coefficient to check correlation between variables and also with dependent variable

And also we check the multicolinearity using VIF and remove those who are having high VIF value.

### 4. Categorical Encoding

### **Encoding**

Encoding is a technique in feature engineering that is used to convert categorical variables into numerical values that can be used by machine learning algorithms.

There are several encoding techniques, including:

1. One-hot encoding: creates a binary column for each unique category, with a value of 1 indicating the presence of the category and 0 indicating the absence.
2. Label encoding: assigns a unique integer value to each category.
3. Ordinal encoding: assigns an ordered integer value to each category based on the natural ordering of the categories.
4. Count encoding: replaces a categorical value with the number of times it appears in the dataset.


In [ ]:
# Converting snowfall and rainfall to categorical attributes
data['snowfall'] = data['snowfall'].apply(lambda x: 1 if x>0 else 0)
data['rainfall'] = data['rainfall'].apply(lambda x: 1 if x>0 else 0)

In [ ]:
# encoding the visibility column
data['visibility'] = data['visibility'].apply(lambda x: 0 if 0<=x<=399 else (1 if 400<=x<=999 else 2))

In [ ]:
# encoding
data['holiday'] = np.where(data['holiday'] == 'Holiday', 1,0)

In [ ]:
cat_var

In [ ]:
# One Hot Encoding

# One-hot encode the 'seasons' feature
seasons_onehot = pd.get_dummies(data['seasons'], prefix='seasons')

# drop the original features
data.drop('seasons', axis=1, inplace=True)

# concatenate the one-hot encoded season feature with the rest of the data
data = pd.concat([data, seasons_onehot], axis=1)

#### What all categorical encoding techniques have you used & why did you use those techniques?

Since there are very few day on which there was snowfall / rainfall, it is in our interest that we convert these columns to binary categorical columns indicating whether there was rainfall / snowfall at that particular hour

For visibility

When

Visibility >= 20 Km ---> Clear (high visibility)

4 Km <= Visibility < 10 Km ---> Haze (medium visibility)

Visibility < 4 Km ---> Fog (low visibility)

Converting visibility based on the above mentioned threshold values. Since they are ordinal, we can encode them as 0 (low visibility), 1 (medium visibility), 2 (high visibility)


For seasons, we use here one hot encoding


### 5. Data Transformation

### **Normalization of Target Variable**

A fundamental component of data mining is data normalization, also known as data pre-processing. It refers to transforming the data, specifically converting the source data into a different format that makes it possible to effectively process the data. Data normalization's primary goal is to reduce or even eliminate duplicate data

#### Do you think that your data needs to be transformed? If yes, which transformation have you used. Explain Why?

In [ ]:
fig, ax = plt.subplots(1,2 , figsize = (15,5))

# Distribution plot of Rented Bike Count
dist =sns.distplot(data['rented_bike_count'],hist=True, ax = ax[0])
dist.set(xlabel = 'Rented Bike Count', ylabel ='Density', title = 'Distribution Plot of Target Variable')

# mean line(blue)
dist.axvline(data['rented_bike_count'].mean(), color='magenta', linestyle='dashed', linewidth=2)
# median line(black)
dist.axvline(data['rented_bike_count'].median(), color='black', linestyle='dashed', linewidth=2)

# Boxplot
box = sns.boxplot(data.rented_bike_count, ax= ax[1], orient='h')
box.set(title = 'Outlier Analysis of Target Variable')
plt.show()

Observation

* The graph above indicates that the Rented Bike Count has a moderate right skewness. Linear regression assumes that the dependent variable has a normal distribution, therefore, to meet this assumption, we need to take some measures to normalize the distribution.
* The boxplot above indicates that there are outliers in the rented bike count column.

In [ ]:
# checking the best transformation for our target variable
fig, axs = plt.subplots(1,3, figsize=(16,4))

sns.distplot(np.log1p(data['rented_bike_count']),kde=True, ax=axs[0])
sns.distplot(np.sqrt(data['rented_bike_count']),kde=True, ax=axs[1])
sns.distplot(np.cbrt(data['rented_bike_count']),kde=True, ax=axs[2])

Observations

* Applying a logarithmic transformation to the dependent variable did not help much as it resulted in a negatively skewed distribution.
* Square root and cube root transformations were attempted, but they did not result in a normally distributed variable.
* Therefore, we will use a square root transformation for the regression as it transformed the variable into a well-distributed form.

In [ ]:
fig, ax = plt.subplots(1,2 , figsize = (15,5))

#  checking square root tranformation in our target variable
dist =sns.distplot(np.sqrt(data['rented_bike_count']), ax = ax[0])
dist.set(xlabel = 'Rented Bike Count', ylabel ='Density', title = 'Distribution Plot of Target Variable in sqrt tranformation')

# mean line
dist.axvline(np.sqrt(data['rented_bike_count']).mean(), color='magenta', linestyle='dashed', linewidth=2)
# median line
dist.axvline(np.sqrt(data['rented_bike_count']).median(), color='black', linestyle='dashed', linewidth=2)

# Boxplot
box = sns.boxplot(np.sqrt(data.rented_bike_count), ax= ax[1], orient = 'h')
box.set(title = 'Outlier Analysis of Target Variable in sqrt tranformation')
plt.show()

Observation

* By applying the square root transformation to the skewed Rented Bike Count, we were able to obtain an almost normal distribution, which is in line with the general rule that skewed variables should be normalized in linear regression.
* We find that there are no outliers in the Rented Bike Count column after applying square root transformation.

**Manipulations done and insights found**
1. We checked for correlation coefficient and found that most of the numerical features are positively correlated to our target variable.
2. From heatmap and correlation coefficient, dew_point_temperature and temperature have a correlation coefficient of 0.91 and dew_point_temperature is less correlated to our target variable hence we dropped dew_point_temperature.
We also did a VIF analysis to remove multi-colinearity and since the VIF factor of 'year' is too large hence we removed the year from our data to build our model.
3. We encoded our categorical features which are necessary for the model to understand. We used one hot encoding for 'seasons' and Numeric encoding for 'holiday' and 'functioning_day'. Other columns are already encoded.
4. To treat our target variable we Applied a logarithmic transformation to the target variable did not help much as it resulted in a negatively skewed distribution, Square root, and cube root transformations were attempted, but they did not result in a normally distributed variable. Therefore, we used a square root transformation for the regression as it transformed the variable into a well-distributed form.

### 6. Data Scaling

In [ ]:
# Scaling your data
features = [i for i in data.columns if i not in ['rented_bike_count']]

In [ ]:
# Scaling your data
scaler = StandardScaler()
X = scaler.fit_transform(data[features])

##### Which method have you used to scale you data and why?

In this we have different independent features of different scale so we have used standard scalar method to scale our independent features into one scale.

### 8. Data Splitting

In [ ]:
# Split your data to train and test. Choose Splitting ratio wisely.\# Split your data to train and test. Choose Splitting ratio wisely.
X = data.drop('rented_bike_count', axis=1)
y= np.sqrt(data['rented_bike_count'])

In [ ]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

print(X_train.shape)
print(X_test.shape)

##### What data splitting ratio have you used and why?

To train the model we have split the data into train and test using train_test_split method

We have split 80% of our data into train and 20% into test.

## ***7. ML Model Implementation***

### ML Model - 1

In [ ]:
# ML Model - 1 Implementation

# Fit the Algorithm

# Predict on the model

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 1 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

### ML Model - 2

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 1 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

#### 3. Explain each evaluation metric's indication towards business and the business impact pf the ML model used.

Answer Here.

### ML Model - 3

In [ ]:
# ML Model - 3 Implementation

# Fit the Algorithm

# Predict on the model

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 3 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

### 1. Which Evaluation metrics did you consider for a positive business impact and why?

Answer Here.

### 2. Which ML model did you choose from the above created models as your final prediction model and why?

Answer Here.

### 3. Explain the model which you have used and the feature importance using any model explainability tool?

Answer Here.

## ***8.*** ***Future Work (Optional)***

### 1. Save the best performing ml model in a pickle file or joblib file format for deployment process.


In [ ]:
# Save the File

### 2. Again Load the saved model file and try to predict unseen data for a sanity check.


In [ ]:
# Load the File and predict unseen data.

### ***Congrats! Your model is successfully created and ready for deployment on a live server for a real user interaction !!!***

# **Conclusion**

Write the conclusion here.

### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***